In [1]:

def simulate_gbm_from_returns(n_years=10, n_scenarios=20, mu=0.07, sigma=0.15, periods_per_year=12, start=100.0):
    '''
    Evolution of an initial stock price using Geometric Brownian Model:
        (S_{t+dt} - S_t)/S_t = mu*dt + sigma*sqrt(dt)*xi,
    where xi are normal random variable N(0,1). 
    The equation for percentage returns above is used to generate returns and they are compounded 
    in order to get the prices.    
    Note that default periods_per_year=12 means that the method generates monthly prices (and returns):
    change to 52 or 252 for weekly or daily prices and returns, respectively.
    The method returns a dataframe of prices and the dataframe of returns.
    '''
    dt = 1 / periods_per_year
    n_steps = int(n_years * periods_per_year)
    
    # from GBM equation for percentage returns, returns have mean = mu*dt and std = sigma*sqrt(dt)
    rets = pd.DataFrame( np.random.normal(loc=mu*dt, scale=sigma*(dt)**(0.5), size=(n_steps, n_scenarios)) )
    
    # compute prices by compound the generated returns
    prices = compound_returns(rets, start=start)
    prices = insert_first_row_df(prices, start)
    
    return prices, rets

In [7]:
import numpy as np
import pandas as pd
import math 

In [8]:
S0 = 50 #初始股票或指数价格
T = 10  #时间
r = 0.05  #短期无风险利率
vol = 0.2  #股票或指数的波动率
#随机种子
np.random.seed(1000000)
gbm_date = pd.date_range(start='01-01-2010',end='01-01-2020',freq='B')
M = len(gbm_date)
I = 1
dt = 1/252 #按照惯例还是默认252个交易日
#布朗运动的标准正态分布size = (M,I)
rand = np.random.standard_normal((M,I))
#计算相关的数据,生成股票或者指数价格的矩阵
S = np.zeros_like(rand)
S[0] = S0
for t in range(1,M):
    S[t] = S[t-1]*np.exp((r-vol**2/2)*dt+vol*rand[t]*math.sqrt(dt))   
gbm = pd.DataFrame(S[:,0],index = gbm_date,columns=["指数价格"])
#生成股票或指数的收益率,年化方差和波动率（这里仍然假设一年的交易日数为252）
gbm["日指数收益率"] = np.log(gbm["指数价格"]/gbm["指数价格"].shift(1))
gbm["年化方差"] = 252*np.cumsum(gbm["日指数收益率"]**2)/np.arange(len(gbm))
gbm["年化波动率"] = np.sqrt(gbm["年化方差"])
#清除空值
gbm = gbm.dropna()

In [12]:
gbm["日指数收益率"].mean()*252

0.0036689774955511864

In [26]:
(gbm["指数价格"]/gbm["指数价格"].shift(1)-1).mean()*252

0.024729799728613257

In [33]:
r=0.202690**2+0.0036689774955511864

In [34]:
r

0.04475221359555119

NameError: name 'erk' is not defined